### Exploratory Data Analysis ###


Batch import libraries

In [3]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
stops = set(stopwords.words('english'))
from nltk.stem.porter import PorterStemmer
p_stemmer = PorterStemmer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV 
%matplotlib inline

Import the dataset


In [5]:
df03=pd.read_csv('03-06-2018.csv')
df02=pd.read_csv('02-06-2018.csv')

df01=pd.read_csv('01-06-2018.csv')

df_jun=pd.concat([df01, df02,df03])

df_jun.shape

(15859, 9)

In [6]:
df31=pd.read_csv('31-05-2018.csv')

df30=pd.read_csv('30-05-2018.csv')

df_may=pd.concat([df30, df31])

df_may.shape

(10547, 9)

**Perform EDA**
1. Drop duplicates within each dataframe and then combine them to form a new dataframe
2. Convert the following temporal transformations:
    a. Format the submission & retrieval times
    b. Calculate the durations of posts by subtrating the retrieval time from the time posted
    c. Format series for day and hour posted
3. Calculate the number of words in each submission


In [7]:
def comb_eda_df(df1,df2):
    import datetime as dt
    def get_day(date):
        return time.strftime('%a')
    def get_hm(time):
        return time.strftime('%H')
    
    print(df1.shape)
    print(df1.id.nunique())
    print(df1.subreddit.value_counts())
    df1=df1.drop_duplicates(subset='id',keep="last")
    print(df1.id.nunique())
    print('')
    

    print(df2.shape)
    print(df2.id.nunique())
    print(df2.subreddit.value_counts())
    df2=df2.drop_duplicates(subset='id',keep="last")
    print(df2.id.nunique())
    print('')
    
    df_new = pd.concat([df1, df2])
    df_new=df_new.drop_duplicates(subset='id',keep="last")
    df_new['Time'] = pd.to_datetime(df_new.created, unit='s')
    df_new['Time_Retrieved'] = pd.to_datetime(df_new.time_retrieved)
    df_new['Post_Duration']=(df_new['Time_Retrieved']-df_new['Time'])
    df_new['Post_Duration']=(df_new['Post_Duration'].dt.seconds/60/60).round(0)
    df_new['Time_HM']= df_new["Time"].apply(get_hm)
    df_new['Day'] =df_new["Time"].apply(get_day)
    df_new['Word_Count'] = [len(x.split()) for x in df_new['title'].tolist()]
    print(df_new.id.nunique())
    print(df_new.info())
    return(df_new)

Combine dataframes and drop duplicates 

In [8]:
df=comb_eda_df(df_may,df_jun
              )

(10547, 9)
5593
politics        3733
Conservative    3558
Liberal         3256
Name: subreddit, dtype: int64
5593

(15859, 9)
6311
politics        5667
Conservative    5287
Liberal         4905
Name: subreddit, dtype: int64
6311



/Users/admin/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



7224
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7224 entries, 12 to 5278
Data columns (total 15 columns):
body              147 non-null object
comms_num         7224 non-null int64
created           7224 non-null float64
domain            7224 non-null object
id                7224 non-null object
score             7224 non-null int64
subreddit         7224 non-null object
time_retrieved    7224 non-null object
title             7224 non-null object
Time              7224 non-null datetime64[ns]
Time_Retrieved    7224 non-null datetime64[ns]
Post_Duration     7224 non-null float64
Time_HM           7224 non-null object
Day               7224 non-null object
Word_Count        7224 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(3), object(8)
memory usage: 903.0+ KB
None


In [23]:
df.duplicated().sum()

0

In [24]:
df.subreddit.value_counts()


politics        3382
Conservative    2184
Liberal         1658
Name: subreddit, dtype: int64

In [26]:
# seperate the politics subreddits from the Conservative & Liberals ones
df_lib_con=df[(df['subreddit']=='Liberal') | (df['subreddit']=='Conservative')] 
df_pol=df[(df['subreddit']=='politics')]
# OR 
# movies[(movies.duration >= 200) | (movies.genre == 'Drama')]

In [27]:
def seperate_dfs(df):
    TodaysDate = time.strftime("%d-%m-%Y")
    csv_name = TodaysDate +".csv"
    df_new.to_csv(csv_name,index=False)
    print(csv_name)